# Employee Churn Analysis

### Purpose
- This notebook explores employee behavior and store data from a cleaned dataset.

### Highlights
1. Overall Yearly Employee Churn Rate Percentage Per Store (SQL)

2. Churn Rate By Department (Pandas)
    - Which departments are struggling the most with employee retention?

3. Tenure vs Churn (Pandas)
    - Which stores have the highest proportion of short, medium, and long tenure employees, and how does tenure relate to overall churn?
    - Tenure Categories
        - Short Service: < 2 years
        - Medium Service: 2–5 years
        - Long Service: > 5 years

4. Churn By Demographics (Pandas)
    - Which gender shows higher churn rates per store?
 

## Setup

In [80]:
# Imports
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(".."))
from etl import get_engine

# Create sqlalchemy engine
engine = get_engine()

# Load tables into dataframes
employee_df: pd.DataFrame = pd.read_sql('SELECT * FROM employees', engine)
store_df: pd.DataFrame = pd.read_sql('SELECT * FROM store', engine)

# Merge tables into a base dataframe
df: pd.DataFrame = employee_df.merge(store_df, on='store_id', how='left')
df.head()

,employee_id,store_id,job_title,age,gender,date_hired,date_terminated,years_employed,snapshot_record_year,employee_status,status_year,city,department
0,1318,35,CEO,52,Male,1989-08-28,None,17,2006-12-31,ACTIVE,2006,Vancouver,Executive
1,1318,35,CEO,53,Male,1989-08-28,None,18,2007-12-31,ACTIVE,2007,Vancouver,Executive
2,1318,35,CEO,54,Male,1989-08-28,None,19,2008-12-31,ACTIVE,2008,Vancouver,Executive
3,1318,35,CEO,55,Male,1989-08-28,None,20,2009-12-31,ACTIVE,2009,Vancouver,Executive
4,1318,35,CEO,56,Male,1989-08-28,None,21,2010-12-31,ACTIVE,2010,Vancouver,Executive


## 1. Overall Yearly Employee Churn Rate Percentage Per Store (SQL)

churn rate = (employees lost in year / total employees at start of year) * 100

In [81]:
# Query 


## 2. Churn Rate By Department (Pandas)
- Which departments are struggling the most with employee retention?

In [ ]:
# Copy to preserve original dataframe
department_copy = df.copy()

# Filter for deparatment and employee_status columns
filtered_df: pd.DataFrame = department_copy.filter(items=['employee_id', 'employee_status', 'department'])

# Total number of unique employees per department
total_employees_df: pd.DataFrame = (
    filtered_df
    .drop_duplicates(subset=['employee_id'])
    .groupby('department')
    .agg(all_employees = ('employee_id', 'nunique'))
    .reset_index()
)

# Filter for terminated employees
terminated: pd.DataFrame = filtered_df.query("employee_status == 'TERMINATED'")

# Count unique number of terminated employees per department - data is longitudinal and don't want to count same employees more than once
terminated_employees_df: pd.DataFrame = (
    terminated
    .groupby('department')
    .agg(terminated_employees = ('employee_id', 'nunique'))
    .reset_index()
)

# Merge two employee tables into one
churn_df: pd.DataFrame = total_employees_df.merge(terminated_employees_df, on='department', how='left').fillna(0)


# Calculate churn rate and sort in descending order
churn_df['department_churn_percentage'] = ((churn_df['terminated_employees'] / churn_df['all_employees']) * 100).round(2)
churn_df.sort_values(by='department_churn_percentage', ascending=False, inplace=True)

## 3. Tenure vs Churn (Pandas)
- Which stores have the highest proportion of short, medium, and long tenure employees, and how does tenure relate to overall churn?


In [ ]:
# Helper Function to maps years_employed into tenure categories
def map_tenure_category(years: int) -> str:
    if years < 2:
        return 'Short'
    elif 2 <= years <= 5:
        return 'Medium'
    return 'Long'

In [ ]:
# Copy to preserve original dataframe
tenure_copy: pd.DataFrame = df.copy()

# Filter for needed columns 
tenure_df: pd.DataFrame = tenure_copy.filter(items=['employee_id', 'employee_status', 'years_employed', 'store_id' ])

# Map employees into tenure categories
tenure_df['tenure_category'] = tenure_df['years_employed'].apply(map_tenure_category)

# Count total number of employees in each tenure_category per store
total_employees_by_tenure_df = (
    tenure_df
    .groupby(['store_id', 'tenure_category'])
    .agg(all_employees=('employee_id', 'nunique'))
    .reset_index()
)

# Count number of terminated employees in each tenure_category per store
terminated_employees_by_tenure_df = (
    tenure_df
    .query("employee_status == 'TERMINATED'")
    .groupby(['store_id', 'tenure_category'])
    .agg(terminated_employees=('employee_id', 'nunique'))
    .reset_index()
)

# Merge employee dataframes
tenure_churn_df: pd.DataFrame = (
    total_employees_by_tenure_df
    .merge(terminated_employees_by_tenure_df, on=['store_id', 'tenure_category'], how='left').fillna(0)
    .assign(
        tenure_category=lambda df: pd.Categorical(      # Convert tenure_category to an ordered categorical so i can sort in order of Short, Medium, Long
            df['tenure_category'],
            categories=['Short', 'Medium', 'Long'],
            ordered=True
        ),
        
        # Cast to int, pandas upcasts column to float because missing values (NaN) are floats and column had NaNs
        terminated_employees= lambda df: df['terminated_employees'].astype(int), 

        # Add column for total employees per store
        total_employees_per_store = lambda df: ( 
            df
            .groupby('store_id')
            ['all_employees'].transform('sum')
        ),

        # Calculate percent of each tenure category per store: (total employees in category) / (total employees in store)
        tenure_percent_per_store = lambda df: ((df['all_employees'] / df['total_employees_per_store']) * 100).round(2),

        # Calculate churn rate per tenure category per store: (terminated employees in category) / (total employees in category).
        category_churn_rate = lambda df: ((df['terminated_employees'] / df['all_employees']) * 100).round(2)
    )
    .sort_values(['store_id', 'tenure_category'])
    .reset_index(drop=True)
)


,store_id,tenure_category,all_employees,terminated_employees,total_employees_per_store,tenure_percent_per_store,category_churn_rate
0,1,Short,36,2,157,22.93,5.56
1,1,Medium,50,2,157,31.85,4.00
2,1,Long,71,14,157,45.22,19.72
3,2,Short,26,1,108,24.07,3.85
4,2,Medium,30,0,108,27.78,0.00


## 4. Churn By Demographics
- Which gender shows higher churn rates per store?

In [162]:
# Copy to preserve original dataframe
gender_copy: pd.DataFrame = df.copy()

# Filter for needed columns
gender_df: pd.DataFrame = gender_copy.filter(items=['employee_id', 'gender', 'employee_status', 'store_id'])

# Count total employees per gender
total_gender_df: pd.DataFrame = (
    gender_df
    .drop_duplicates()
    .groupby(['store_id', 'gender'])
    .agg(total_count = ('employee_id', 'count'))
    .reset_index()
)

# Count total terminated employees per gender
terminated_gender_df: pd.DataFrame = (
    gender_df
    .query("employee_status == 'TERMINATED'")
    .drop_duplicates()
    .groupby(['store_id', 'gender'])
    .agg(terminated_count = ('employee_id', 'count'))
    .reset_index()
)

# Merge both dataframes
gender_churn_df: pd.DataFrame = total_gender_df.merge(terminated_gender_df, on=['store_id', 'gender'], how='left').fillna(0)

# Calculate churn rate per gender per store 
gender_churn_df['churn_rate'] = ((gender_churn_df['terminated_count'] / gender_churn_df['total_count']) * 100).round(2)

gender_churn_df.head()





,store_id,gender,total_count,terminated_count,churn_rate
0,1,Female,57,16,28.07
1,1,Male,47,2,4.26
2,2,Female,41,10,24.39
3,2,Male,34,1,2.94
4,3,Female,14,3,21.43
